In [ ]:
%load_ext autoreload
%autoreload 2

In [3]:
import json
import logging

logging.basicConfig(level=logging.WARNING)

from src.data.VulnerabilityReport import create_from_flama_json # this is the function that creates the VulnerabilityReport object from the json data
from src.ai.LLM.LLMServiceStrategy import LLMServiceStrategy 

# We need the json data, so we are loading it from the file
with open('../data/Dataset.json') as f:
    data = json.load(f)
    
from src.ai.LLM.Stretegies.OpenAIService import OpenAIService
from src.ai.LLM.Stretegies.OLLAMAService import OLLAMAService

my_strategy = OpenAIService()
llm_service = LLMServiceStrategy(my_strategy)
my_strategy_local = OLLAMAService()
llm_service_local = LLMServiceStrategy(my_strategy)

n=5 # number of findings to import
vulnerability_report = create_from_flama_json(data, n = n, llm_service=llm_service) # here, we create the VulnerabilityReport object, consisting of a list of Finding objects which each have Solution objects. We pass the llm_service to the VulnerabilityReport object, but this can also just be omitted, in which case it will be created in each Finding object.
vulnerability_report.sort() # this will sort the findings by severity

In [4]:
vulnerability_report.set_llm_service(llm_service_local)
vulnerability_report.add_category() # this will add categories to the findings

100%|██████████| 5/5 [00:02<00:00,  1.70it/s]


In [6]:
vulnerability_report.set_llm_service(llm_service)
vulnerability_report.add_solution(long=True, short=True, search_term=True) # this will add solutions to the findings

100%|██████████| 5/5 [02:26<00:00, 29.25s/it]


In [ ]:
with open(f'../data/VulnerabilityReport_{n if n !=-1 else "all"}_' + my_strategy.get_model_name().replace(":","_") +'.json', 'w') as f:
    f.write(str(json.dumps(vulnerability_report.to_dict())))

In [ ]:
from IPython.display import HTML, Markdown
Markdown(vulnerability_report.to_html(table=True))


In [ ]:
HTML(str(vulnerability_report).replace('\n', '<br />'))
